# 0 - Environment Set Up 

In [ ]:
# Install required packages
!pip install -r ../requirements.txt -q

# 1 - Project Set Up

## 1.1 - Map to Root

In [ ]:
REL_PATH_TO_ROOT = "../"

In [ ]:
import sys
import os

In [ ]:
sys.path.insert(0,REL_PATH_TO_ROOT)

In [ ]:
from src.utils import get_root_dir, test_root_dir
from local_variables import ROOT_DIR

In [ ]:
test_root_dir(REL_PATH_TO_ROOT)

## 1.2 - Set-Up Imports

In [ ]:
from prompt_manager.manager import PromptManager
from prompt_manager.fetcher import fetch_prompt
from src.api import generate_outputs_openai

In [ ]:
import pandas as pd
import random as rand

## 1.3 - Data Imports

In [ ]:
# Bring in dataset of articles
summar_df = pd.read_csv(f"{get_root_dir()}/data/summarization.csv",index_col=0)

In [ ]:
summar_df.head()

# 2 - Generating Ground Truths

## 2.1 - Choice Model

In [ ]:
# Specify metric names for summarisation
METRICS = ["fluency","brevity","coverage"]

In [ ]:
# Initialise ground truth columns for these metrics in the dataframe
summar_df[METRICS] = ""

In [ ]:
# Create a random generator that decides whether or not there should be an error of a given metric type
def ground_truth_generation(metric_list):
    metric_ground_truths = {metric : bool(rand.randint(0,1)) for metric in metric_list}
    return metric_ground_truths

In [ ]:
# Example output, false = no error, true = error present
ground_truth_generation(METRICS)

In [ ]:
# Here is the base prompt we use to generate the summary from the article

# ARTICLE is the placeholder for the article to be summarised

# Each rule is an instruction to the AI to either obey the rule or to break it, see prompts in prompts/dev/summarisation_prompts
core_prompt = """

Your job is a summariser, working for a special organisation. Your job is to write a SUMMARY for the ARTICLE, however you must follow SPECIAL RULES set by the organisation. 

Your summary is constrained by 3 Rules: FLUENCY, BREVITY, COVERAGE. When writing the summary obey the rules as stipulated in the RULES section:

**ARTICLE**:
{ARTICLE}

**BREVITY RULE**
Brevity: {brev_rule}

**COVERAGE RULE**
Coverage: {cov_rule}

**FLUENCY RULE**:
Fluency: {flu_rule}

WRITE YOUR SUMMARY FOLLOWING THE RULES ABOVE

Return your summary:

Chop chop!
"""

In [ ]:
# Make a map from the metrics themselves to their rule placeholders
metric_to_core_prompt_placeholder ={"fluency" : "{flu_rule}", "brevity" : "{brev_rule}", "coverage" : "{cov_rule}"}

In [ ]:
# Import rules to make errors or avoid them (error or unerror respectively)
from prompts.dev.summarisation_prompts import error_prompts, unerror_prompts

In [ ]:
# Create choice engine

def choice_engine(core_prompt, metric_list,ground_truth, article, error_prompts, unerror_prompts, metric_to_core_prompt_placeholder, article_placeholder="{ARTICLE}"):
    """
    Generate system prompt for OpenAI response with summary of an article with known errors (or lack thereof) based on desired ground truth

    Args:
        core_prompt (str): system prompt used to generate OpenAi response.
        metric_list (list): list of metrics for summarisation
        ground_truth (dict): dictionary of metric list values as keys and the values being whether or not an error should be introduced for that metric (True - error, False - no error)
        article (str): text of the article to be summarised
        error_prompts (dict): list of prompts corresponding to known errors for the summarisation metrics (same keys as metric_list values)
        unerror_prompts (dict): list of prompts corresponding to avoiding errors for the summarisation metrics (same keys as metric_list values)
        metric_to_core_prompt_placeholder (dict): specify a map between the values in metric_list and the placeholder for the (un)error prompts to be substituted into the core prompt
        article_placeholder (str): name of the placeholder in the core prompt where the article should be substituted 

    Returns:
        core_prompt: returns core_prompt with appropriate formatting absed on ground truth
        """
    
    core_prompt = core_prompt.replace(article_placeholder,article)

    for key, error_bool in zip(ground_truth.keys(), ground_truth.values()):
        placeholder = metric_to_core_prompt_placeholder[key]
        
        if error_bool:
            metric_rule = error_prompts[key]
        else:
            metric_rule = unerror_prompts[key]

        core_prompt = core_prompt.replace(placeholder,metric_rule)

    return core_prompt

In [ ]:
evaluator_responses = []

summar_df = summar_df.sample(n=30,random_state=42)

# Loop through dataset
for index, row in summar_df.iterrows():

    # Get row level inputs
    article = row["article"]

    # Get random ground truth for this row
    ground_truth = ground_truth_generation(METRICS)

    # Loop through the ground truth metrics and assign the ground truth of an error to the row in the dataframe
    for key in ground_truth.keys():
        summar_df.at[index, key] = ground_truth[key]
   

    # Format prompt for ground truth
    prompt = choice_engine(core_prompt,METRICS,ground_truth,article,error_prompts,unerror_prompts,metric_to_core_prompt_placeholder)
    
    # Send prompt and collect response
    response = generate_outputs_openai(prompt)
    evaluator_responses.append(response)


summar_df["altered_summaries"] = evaluator_responses

In [ ]:
saved_copy = summar_df.copy(deep=False)

In [ ]:
# Ensure grammar errors but relooping

fluency_error_prompt = """Introduce spelling, grammar and punctuation errors in the following TEXT:
{TEXT_PLACEHOLDER}

Return the errored text below:
"""

# Loop through dataset
for index, row in summar_df.iterrows():

    # Get row level inputs
    summary = row["altered_summaries"]

    # Get random ground truth for this row
    ground_truth = row["fluency"]

    # Loop through the ground truth metrics and assign the ground truth of an error to the row in the dataframe
    if ground_truth:
        prompt = fluency_error_prompt.replace("TEXT_PLACEHOLDER",summary)
        # Send prompt and collect response
        response = generate_outputs_openai(prompt)
        summar_df.at[index, 'altered_summaries'] = response

In [ ]:
processed_df = summar_df[["article","fluency","brevity","coverage","altered_summaries"]].copy(deep=False)

In [ ]:
processed_df = processed_df.rename(columns={"article" : "original_article", "altered_summaries" : "journalist_mini_summary", "fluency" : "is_fluency_error_ground_truth", "brevity" : "is_brevity_error_ground_truth", "coverage" : "is_coverage_error_ground_truth"})

In [ ]:
processed_df.to_csv(f"{get_root_dir()}/data/summarisation.csv")